<a href="https://colab.research.google.com/github/ru-corporate/boo/blob/master/boo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip uninstall boo -y
!pip install --upgrade git+https://github.com/ru-corporate/boo.git@master

In [0]:
!pip install --upgrade boo

In [3]:
import boo
YEAR = 2018
boo.download(YEAR)
boo.build(YEAR)

Already downloaded: /root/.boo/raw2018.csv (1549.0M)
Use download(2018, force=True) to overwrite existing file.
Already built: /root/.boo/2018.csv (685.4M)
Use build(2018, force=True) to overwrite existing file.


In [0]:
!ls -s ~/.boo

total 2288064
 701880 2018.csv  1586184 raw2018.csv


In [0]:
boo.download(1)
boo.build(1)
%timeit boo.main.read_intermediate_df(1) 
df = boo.main.read_intermediate_df(1)

3 k [00:00, 1095.31 k/s]
15 lines [00:00, 11832.72 lines/s]

Saved as /root/.boo/raw1.csv (0.0M)
Reading from /root/.boo/raw1.csv (0.0M)
Saving to /root/.boo/1.csv (does not exist)
Saved /root/.boo/1.csv (0.0M)


100 loops, best of 3: 5.88 ms per loop


In [4]:
import boo
from boo.dataframe.canonic import (adjust_rub, more_columns, rename_rows, 
                                   canonic_columns,
                                   add_okved_subcode, add_region, add_title)
                                   

def canonic_df(df):
    """Преобразовать данные внтури датафрейма:

    - Привести все строки к одинаковым единицам измерения (тыс. руб.)
    - Убрать  неиспользуемые колонки (date_revised, report_type)
    - Новые колонки:
        * короткое название компании
        * три уровня кода ОКВЭД
        * регион (по ИНН)

    """
    df_ = add_okved_subcode(add_region(add_title(df)))
    df_ = rename_rows(df_)
    df_ = adjust_rub(df_)
    return df_[canonic_columns()].set_index('inn')

print("obtaining source...")
root_df0 = boo.main.read_intermediate_df(YEAR)

print("canonic_df(df)")
df = root_df0.copy()
%timeit canonic_df(df)  

print("columns")
df = root_df0.copy()
%timeit add_okved_subcode(add_region(add_title(df)))

print("adjust rub")
df = root_df0.copy()
%timeit adjust_rub(df)

print("renaming")
df = root_df0.copy()
%timeit rename_rows(df)

#%timeit boo.main.read_intermediate_df(2017)
#%timeit boo.dataframe.canonic.adjust_rub(root_df)
#%timeit boo.main.read_dataframe(2017)

#%timeit adjust_rub(root_df)
#%timeit more_columns(root_df)
#%timeit add_okved_subcode(root_df)
#%timeit add_region(root_df)
#%timeit add_title(root_df)
#%timeit rename_rows(root_df)
#%timeit root_df[canonic_columns()].set_index('inn')

obtaining source...
canonic_df(df)


KeyError: ignored

In [0]:
import pandas as pd
import boo

zf = pd.DataFrame(dict(title=["a", "a"]), index=['333', '4716016979'])

def rename_rows2(df):
    df = df.copy(deep=True)
    RENAME_DICT = {
        '2460066195': "РусГидро",
        '4716016979': "ФСК ЕЭС",
        '7702038150': "Московский метрополитен",
        '7721632827': "Концерн Росэнергоатом",
        '7706664260': "Атомэнергопром",
        '7703683145': "Холдинг ВТБ Капитал АЙ БИ",
        '9102048801': "Черноморнефтегаз",
        '7736036626': "РИТЭК"
    }
    for k, v in RENAME_DICT.items():
        df.loc[k, 'title'] = v
    return df

#print("obtaining source...")
#root_df0 = boo.main.read_intermediate_df(2017)

#print("renaming")
#df = root_df0.copy()
#%timeit rename_rows2(df)
#del df
x = rename_rows2(zf)
x

In [0]:
def medium_companies(df):
    _df = df.query("sales > 1_000") \
            .sort_values("sales", ascending=False)
    return to_mln(_df)
    

def shorthand(df):
    return df.rename(columns={'profit_before_tax': 'p',
                              'profit_before_tax_lag': 'p_lag',
                              'tp_capital': 'cap',
                              'tp_capital_lag': 'cap_lag'})

In [0]:
import boo
from boo.dataframe.util import industry, inn, contains, sort
from boo.dataframe.filter import (large_companies, medium_companies,
                                  minimal_columns, is_alive, shorthand, 
                                  to_mln, to_bln)

try:
  root_df   
except NameError:
  root_df = boo.read_dataframe(2017)

del df  
try:   
  df
except NameError:
  df = shorthand(minimal_columns(large_companies(root_df)))


print("Исходное количество компаний ():               %7i" % root_df.shape[0])
print("Действующие крупные компании (после фильтров): %7i" % df.shape[0])

In [0]:
df2 = shorthand(medium_companies(root_df)) 

In [0]:
df2.sample(3)['region ok1 ok2 title ta sales p cf'.split(' ')]

In [0]:
contains(df2, "твое")['title sales ok1'.split(' ')]
industry(df2, 46)
#inn(df2, 7743238687)

In [0]:
ix = [7717582404, 7709770002, 7702832965,7705440959, 5047085094, 7731522580, 
      7729355029, 7706741162,
      7707099460, 7704256932,4825037471,7706202481, 7705840940,
      6166034397, 6166019871
     ] 
cols = ['title','ta','sales', 'sales_lag', 'p', 'p_lag', 'cf_inv', 'paid_fa_investment']
hh = inn(df2, ix)[cols].rename(columns=dict(paid_fa_investment='fi'))
hh['r'] = (hh.p / hh.sales *100).round(1)
hh['r_lag'] = (hh.p_lag / hh.sales_lag *100).round(1)
hh['g'] = (hh.sales / hh.sales_lag - 1).round(2) * 100
hh['i'] = -(hh.cf_inv / hh.ta).round(2) * 100
hh.sort_values('sales', ascending = False)

In [0]:
root_df.columns


# Примеры выборок

In [0]:
df.query('ta>100').sample(10)

In [0]:
# Какие компании не попадают в основные отрасли?
exclude = [64,66,46,6,24,49,19,20,35,47,45,29,41,42,61,68]
sort(df, "ta")[~df.ok1.isin(exclude)].iloc[:20,:]

In [0]:
inn(df, 1660056570)

In [0]:
for i in [41, 42]:
  print(i, boo.okved.name_v2(i))

In [0]:
industry(df, 42).head(10)

In [0]:
contains(df, "КРОКУС ИНТЕРНЭШНЛ")

In [0]:
contains(df, "Стройтрансгаз")

In [0]:
contains(df, "Стройгазмонтаж")

# Компании по отраслям 

In [0]:
for i in boo.okved.all_codes_v2():
   x = industry(df.query('ta>5'), i).head(20)
   if not x.empty:
      print (i, boo.okved.name_v2(i))
      print (x.to_string(line_width=200))

In [0]:
industry(df,56,10) 

In [0]:
contains(df, "бетон")
industry(df, 23, 51)